# Importar librerias

In [1]:
import torch
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # NVIDIA GeForce RTX 3050

True
NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os, math, random, pathlib
from typing import Tuple
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms as T

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

# Dimensionar las imagenes

In [6]:
from torchvision import transforms

# Transformaciones para entrenamiento (aumentos + normalización)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=0,
                            translate=(0.05, 0.05)),  # equivalente a width/height_shift
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# Transformaciones para validación y test (solo reescalado + normalización)
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# Inicio del codigo

In [7]:
# ----------------- Config -----------------
BASE_DIR   = "002_cancer"
TRAIN_DIR  = os.path.join(BASE_DIR, "train")
VAL_DIR    = os.path.join(BASE_DIR, "val")
TEST_DIR   = os.path.join(BASE_DIR, "test")

CLASSES = 2
BATCH   = 32
ROWS = COLS = 224
INPUT_CH = 3
SEED         = 42
EPOCHS       = 15
TEST_MAX_SAMPLES = 3000

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
torch.manual_seed(SEED)


Using device: cuda


In [8]:
# ===== Datasets & Dataloaders =====
from torch.utils.data import random_split

# Comprobación de carpetas
for p in [TRAIN_DIR, TEST_DIR]:
    assert os.path.isdir(os.path.join(p, "Benign")),    f"Falta {p}/Benign"
    assert os.path.isdir(os.path.join(p, "Malignant")), f"Falta {p}/Malignant"

use_explicit_val = os.path.isdir(VAL_DIR) and all(
    os.path.isdir(os.path.join(VAL_DIR, c)) for c in ["Benign", "Malignant"]
)

if use_explicit_val:
    train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
    val_ds   = datasets.ImageFolder(VAL_DIR,   transform=val_test_transforms)
else:
    # Split desde TRAIN -> (train, val)
    full_train = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
    val_ratio  = 0.2
    n_total    = len(full_train)
    n_val      = int(n_total * val_ratio)
    n_train    = n_total - n_val
    train_ds, val_ds = random_split(
        full_train, [n_train, n_val],
        generator=torch.Generator().manual_seed(SEED)
    )

test_ds  = datasets.ImageFolder(TEST_DIR, transform=val_test_transforms)

NUM_WORKERS = 0 if os.name == "nt" else 4
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=(device=="cuda"))
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=(device=="cuda"))
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=(device=="cuda"))

# Mapeo de clases
if use_explicit_val:
    class_to_idx = datasets.ImageFolder(TRAIN_DIR).class_to_idx
else:
    class_to_idx = train_ds.dataset.class_to_idx if hasattr(train_ds, "dataset") else datasets.ImageFolder(TRAIN_DIR).class_to_idx
idx_to_class = {v:k for k,v in class_to_idx.items()}
print("Clases:", class_to_idx)
print(f"Train={len(train_ds)} | Val={len(val_ds)} | Test={len(test_ds)}")


Clases: {'Benign': 0, 'Malignant': 1}
Train=9504 | Val=2375 | Test=2000


In [9]:
# ===== Modelo básico (CNN pequeña) =====
import torch.nn as nn
import torch.nn.functional as F

class BasicCNN(nn.Module):
    def __init__(self, in_ch=3, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_ch, 32, kernel_size=3, padding=1),  # [B,32,224,224]
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                                  # [B,32,112,112]

            nn.Conv2d(32, 64, kernel_size=3, padding=1),      # [B,64,112,112]
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                                  # [B,64,56,56]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),     # [B,128,56,56]
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                                  # [B,128,28,28]

            nn.Conv2d(128, 256, kernel_size=3, padding=1),    # [B,256,28,28]
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                                  # [B,256,14,14]
        )
        self.pool = nn.AdaptiveAvgPool2d((1,1))               # [B,256,1,1]
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

model = BasicCNN(in_ch=INPUT_CH, num_classes=CLASSES).to(device)
print(model)


BasicCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, aff

In [10]:
# ===== Entrenamiento =====
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

# Bandera para AMP: True solo si hay GPU
USE_AMP = (device == "cuda")
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)

best_val_loss = float("inf")
best_path = "basic_cnn_best.pt"
patience = 5
epochs_no_improve = 0

def run_epoch(loader, train_mode=True):
    if train_mode:
        model.train()
    else:
        model.eval()
    total, correct, running = 0, 0, 0.0

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)

        if train_mode:
            optimizer.zero_grad(set_to_none=True)

        if USE_AMP:
            with torch.cuda.amp.autocast():
                logits = model(xb)
                loss   = criterion(logits, yb)
        else:
            logits = model(xb)
            loss   = criterion(logits, yb)

        if train_mode:
            if USE_AMP:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

        running += loss.item() * xb.size(0)
        pred = logits.argmax(1)
        total += yb.size(0)
        correct += (pred == yb).sum().item()

    return running/total, correct/total


C:\Users\Usuario\AppData\Local\Temp\ipykernel_20572\772398189.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)


In [11]:
# ===== Loop de entrenamiento con early stopping =====
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, train_mode=True)
    with torch.no_grad():
        va_loss, va_acc = run_epoch(val_loader, train_mode=False)

    scheduler.step(va_loss)

    print(f"[{epoch:02d}/{EPOCHS}] "
          f"train_loss={tr_loss:.4f} acc={tr_acc:.3f} | "
          f"val_loss={va_loss:.4f} acc={va_acc:.3f} | "
          f"LR={optimizer.param_groups[0]['lr']:.6f}")

    # early stopping
    if va_loss < best_val_loss - 1e-4:
        best_val_loss = va_loss
        epochs_no_improve = 0
        torch.save({"model": model.state_dict(), "class_to_idx": class_to_idx}, best_path)
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping.")
            break
        

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20572\772398189.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


[01/15] train_loss=0.4479 acc=0.785 | val_loss=0.3795 acc=0.822 | LR=0.001000
[02/15] train_loss=0.4004 acc=0.814 | val_loss=0.3514 acc=0.829 | LR=0.001000
[03/15] train_loss=0.3704 acc=0.829 | val_loss=0.3438 acc=0.851 | LR=0.001000
[04/15] train_loss=0.3493 acc=0.842 | val_loss=0.3302 acc=0.855 | LR=0.001000
[05/15] train_loss=0.3430 acc=0.848 | val_loss=0.3301 acc=0.868 | LR=0.001000
[06/15] train_loss=0.3227 acc=0.858 | val_loss=0.3180 acc=0.869 | LR=0.001000
[07/15] train_loss=0.3237 acc=0.859 | val_loss=0.3075 acc=0.873 | LR=0.001000
[08/15] train_loss=0.3173 acc=0.862 | val_loss=0.3223 acc=0.865 | LR=0.001000
[09/15] train_loss=0.3116 acc=0.864 | val_loss=0.3116 acc=0.865 | LR=0.001000
[10/15] train_loss=0.3131 acc=0.862 | val_loss=0.2946 acc=0.879 | LR=0.001000
[11/15] train_loss=0.3091 acc=0.867 | val_loss=0.3076 acc=0.867 | LR=0.001000
[12/15] train_loss=0.3023 acc=0.869 | val_loss=0.3123 acc=0.872 | LR=0.001000
[13/15] train_loss=0.2993 acc=0.869 | val_loss=0.2920 acc=0.875 